In [1]:
import os
import cv2
import torch
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [2]:
model_type = "DPT_Large"
depth_estimator = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
depth_estimator.to(device)
depth_estimator.eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

Using cache found in C:\Users\Vibhor/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\Vibhor/.cache\torch\hub\intel-isl_MiDaS_master


In [3]:
path = "epipolarImages/"
lst = os.listdir(path)
cmap = plt.get_cmap('jet')

if not os.path.exists("depth_maps"):
    os.mkdir("depth_maps")
else:
    for f in os.listdir("depth_maps"):
        os.remove(os.path.join("depth_maps", f))

imgList = os.listdir(path)
if not os.path.exists("depth_maps"):
    os.mkdir("depth_maps")

for image in imgList:
    img = cv2.imread(path + "/"+ image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # reshape the max dimension to 720px and resize the other dimension accordingly
    scale = 720 / max(img.shape[0], img.shape[1]) 
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    img = cv2.resize(img, (w, h))

    input_batch = transform(img).to(device)
    with torch.no_grad():
        mask = depth_estimator(input_batch)
        mask = torch.nn.functional.interpolate(
            mask.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    mask = mask.cpu().numpy()
    cv2.imwrite("depth_maps/bw_" + image, mask)
    mask = cmap(mask / np.max(mask))[:,:,[2,1,0]] * 255
    mask = mask.astype(np.uint8)
    cv2.imwrite("depth_maps/" + image, mask)